In [10]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# FIX ME: Root dir as argument
root_dir = "../../data/picklerCheckpoints/"
is_compressed = True;


In [11]:
import numpy as np
import pandas as pd
import nbimporter
import geopandas as gpd
import commons.utility.pickle_util as pickle
import Preprocessing.DataEnrichment as de
from shapely.geometry import Point

coord_system = {'init': 'epsg:4326'}


In [12]:
# Import for spatial mapping operations
import bokeh, bokeh.plotting, bokeh.models
from bokeh.io import output_notebook, show
output_notebook()
import sklearn.preprocessing
from shapely.geometry import Point
import rtree


Loading BokehJS ...

In [13]:
data = de.getUberDataforMonth('apr')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Loading file ../data/picklerCheckpoints/enrichedData/uber-enriched-data-apr14.p
Decompressing ../data/picklerCheckpoints/enrichedData/uber-enriched-data-apr14.p.pbz2..............Done
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Dataset loaded


In [14]:
def loadNyCordData():
    nyCoordinates = gpd.read_file('../data/NewYorkCoords/taxi_zones.shp').to_crs(coord_system)
    nyCoordinates = nyCoordinates.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)
    return nyCoordinates

In [15]:
def generateBoundingBoxes(df):
    df2 = df.copy()
    df2['geometry'] = df.geometry.envelope
    df2['borough_categ'] = sklearn.preprocessing.LabelEncoder().fit_transform(df2['borough'])
    return df2

In [16]:
def displayBoundingBoxes(df):
    gjds2 = bokeh.models.GeoJSONDataSource(geojson=df.to_json())
    TOOLS = "pan,wheel_zoom,reset,hover,save"

    p = bokeh.plotting.figure(title="NYC Taxi Districts Bounding Boxes", tools=TOOLS,
        x_axis_location=None, y_axis_location=None)

    color_mapper = bokeh.models.LinearColorMapper(palette=bokeh.palettes.Viridis256)

    p.patches('xs', 'ys', 
              fill_color={'field': 'borough_categ', 'transform': color_mapper},
              fill_alpha=0.7, line_color="black", line_width=0.5,          
              source=gjds2)

    p.grid.grid_line_color = None

    hover = p.select_one(bokeh.models.HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = u"""
    <div> 
        <div class="bokeh_hover_tooltip">Name : @zone</div>
        <div class="bokeh_hover_tooltip">Borough : @borough</div>
        <div class="bokeh_hover_tooltip">Zone ID : @LocationID</div>
        <div class="bokeh_hover_tooltip">(Lon, Lat) : ($x ˚E, $y ˚N)</div>
    </div>
    """

    p.circle([-73.966,], [40.78,], size=10, fill_color='magenta', line_color='yellow', line_width=1, alpha=1.0)

    show(p)

In [19]:
def spatialJoin(bbxDf, dataDf):
    gdf = gpd.GeoDataFrame(dataDf, geometry=gpd.points_from_xy(dataDf.Lon, dataDf.Lat), crs={'init': 'epsg:4326'})
    mappedData = gpd.sjoin(gdf, 
          bbxDf,
          how='left', op='within')
    return mappedData

In [20]:
def mapCoordinatesToNY(df):
    pickleStore = "../data/picklerCheckpoints/spatialJoinData/"
    path = pickleStore + 'NY_boundingBoxData.p'
    if (os.path.exists(path) or os.path.exists(path+'.pbz2')):
        bbxdf = pickle.load(path, True)
        displayBoundingBoxes(bbxdf)
        return spatialJoin(bbxdf, df)
    else:
        nyCoordinates = loadNyCordData()
        bbxdf = generateBoundingBoxes(nyCoordinates)
        pickle.save(bbxdf, pickleStore, 'NY_boundingBoxData', True)
        displayBoundingBoxes(bbxdf)
        return spatialJoin(bbxdf, df)

In [47]:
def run(months):
    validMonths = ['apr', 'may', 'jun', 'jul','aug', 'sep']
    pickleStore = "../data/picklerCheckpoints/spatialJoinData/"
    globalPath = pickleStore + 'uber-spatial-data-14.p'
    toMerge = []
    if(all(i in months for i in validMonths)):
        if (os.path.exists(globalPath) or os.path.exists(globalPath+'.pbz2')):
            print("Spatial join of all months found...Loading...")
            return pickle.load(globalPath, True)
        
    for month in months:
        data = None
        path = pickleStore + 'uber-spatial-data-'+ month +'14.p'
        if(month in validMonths):
            if (os.path.exists(path) or os.path.exists(path+'.pbz2')):
                print("Data for the month :"+ month + " Found.....")
                toMerge.append(pickle.load(path, True))
            else:
                print("Data for the month :"+ month + " is Missing...")
                print("Regenerating data.....")
                data = de.getUberDataforMonth(month)
                sptialJoinData = mapCoordinatesToNY(data)
                pickle.save(sptialJoinData, pickleStore, 'uber-spatial-data-'+month+'14', True)
                toMerge.append(sptialJoinData)
        
    mergedData = pd.concat(toMerge).dropna()
    if (months.sort() == validMonths.sort()):
        pickle.save(mergedData, pickleStore, 'uber-spatial-data-14', True)
        
    return mergedData

In [49]:
outDf = run(['apr', 'may'])

Data for the month :apr Found.....
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Loading file ../data/picklerCheckpoints/spatialJoinData/uber-spatial-data-apr14.p


KeyboardInterrupt: 

In [ ]:
outDf